# MathLang 多次元計算（シナリオ）テスト

このノートブックでは、MathLangの新しい「多次元計算（シナリオ）」機能をテストします。
シナリオ機能を使うと、複数の変数割り当て（コンテキスト）に対して同時に計算・検証を行うことができます。

## テスト内容
1. DSLによるシナリオ定義
2. 複数シナリオでの一括検証
3. 特定のシナリオでのみ成り立つ式の検出（不完全な正解の排除）

In [1]:
# パスの設定
import sys
from pathlib import Path

project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 必要なモジュールをインポート
from core.parser import Parser
from core.evaluator import Evaluator
from core.core_runtime import CoreRuntime
from core.symbolic_engine import SymbolicEngine
from core.computation_engine import ComputationEngine
from core.validation_engine import ValidationEngine
from core.hint_engine import HintEngine

print("モジュールのインポート完了")

モジュールのインポート完了


In [2]:
# ランタイムの初期化
symbolic_engine = SymbolicEngine()
computation_engine = ComputationEngine(symbolic_engine)
validation_engine = ValidationEngine(computation_engine)
hint_engine = HintEngine(computation_engine)

runtime = CoreRuntime(
    computation_engine=computation_engine,
    validation_engine=validation_engine,
    hint_engine=hint_engine
)

print("CoreRuntime 初期化完了")

CoreRuntime 初期化完了


In [3]:
def run_dsl(dsl_text):
    print("=" * 40)
    print("DSL入力:")
    print("-" * 40)
    print(dsl_text.strip())
    print("=" * 40)
    
    # %MathLang ヘッダーがあれば削除
    lines = [line for line in dsl_text.splitlines() if not line.strip().startswith("%MathLang")]
    clean_dsl = "\n".join(lines)
    
    try:
        # パース
        print("\n1. パース中...")
        parser = Parser(clean_dsl)
        program = parser.parse()
        print("   ✓ パース成功")
        
        # 実行
        print("\n2. 実行中...")
        # ランタイムのリセット（簡易的）
        runtime._scenarios = {}
        runtime._current_expr = None
        runtime._context = {}
        
        evaluator = Evaluator(program, runtime)
        success = evaluator.run()
        
        print(f"   {'✓ 実行成功' if success else '✗ 実行失敗'}")
        
    except Exception as e:
        print(f"\n✗ エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()

In [4]:
# テストケース1: 正常なシナリオ検証
# x + x = 2x はどのような x でも成り立つ
dsl_case1 = """
%MathLang
scenario "Positive":
    x = 5
scenario "Negative":
    x = -5
scenario "Zero":
    x = 0

problem: x + x
step: 2*x
end: done
"""

run_dsl(dsl_case1)

DSL入力:
----------------------------------------
%MathLang
scenario "Positive":
    x = 5
scenario "Negative":
    x = -5
scenario "Zero":
    x = 0

problem: x + x
step: 2*x
end: done

1. パース中...
   ✓ パース成功

2. 実行中...
   ✓ 実行成功


In [5]:
# テストケース2: 特定のシナリオでのみ成り立つ式（間違い）
# x + x = x*x は x=0, x=2 でのみ成り立つ
dsl_case2 = """
%MathLang
scenario "Case 2":
    x = 2
scenario "Case 3":
    x = 3

problem: x + x
step: x*x  # x=2では正しいが、x=3では間違い（2+2=4, 2*2=4 vs 3+3=6, 3*3=9）
end: done
"""

run_dsl(dsl_case2)

DSL入力:
----------------------------------------
%MathLang
scenario "Case 2":
    x = 2
scenario "Case 3":
    x = 3

problem: x + x
step: x*x  # x=2では正しいが、x=3では間違い（2+2=4, 2*2=4 vs 3+3=6, 3*3=9）
end: done

1. パース中...
   ✓ パース成功

2. 実行中...
   ✓ 実行成功
